<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/Modelo_Recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SparkSession - é a porta de entrada do PySpark. Ele cria a “sessão” que conecta o código ao cluster Spark (ou roda localmente, se não houver cluster).
# functions as F - permite usar funções SQL já prontas do Spark (ex: F.col, F.avg, F.count, etc.).
from pyspark.sql import SparkSession, functions as F

# ALS (Alternating Least Squares) é o algoritmo de recomendação baseado em fatoração de matrizes. Ele gera embeddings (representações numéricas) de usuários e itens para prever notas ou preferências
from pyspark.ml.recommendation import ALS

# Esse transformador converte valores categóricos em números inteiros.
# Exemplo: Usuário "Caroline" vira 0, "João" vira 1, etc.
# Isso é importante porque o ALS só funciona com IDs numéricos, não strings.
from pyspark.ml.feature import StringIndexer

# Serve para medir o desempenho do modelo de recomendação, calculando métricas como RMSE (Root Mean Square Error).
from pyspark.ml.evaluation import RegressionEvaluator

# .builder - inicia a configuração da sessão.
# .appName("RecomendadorALS") - dá um nome ao app (vai aparecer no monitor do Spark).
# .getOrCreate() - cria a sessão Spark se não existir, se existir, apenas retorna a mesma.
spark = SparkSession.builder.appName("RecomendadorALS").getOrCreate()


In [2]:
# Simulando um conjunto de avaliações
dados = [
    ("Alice", "Filme1", 5.0),
    ("Alice", "Filme2", 3.0),
    ("Bob", "Filme1", 4.0),
    ("Bob", "Filme3", 2.0),
    ("Carol", "Filme2", 4.0),
    ("Carol", "Filme3", 5.0),
]

# Criar DataFrame com colunas nomeadas
df = spark.createDataFrame(dados, ["usuario", "item", "nota"])
df.show()


+-------+------+----+
|usuario|  item|nota|
+-------+------+----+
|  Alice|Filme1| 5.0|
|  Alice|Filme2| 3.0|
|    Bob|Filme1| 4.0|
|    Bob|Filme3| 2.0|
|  Carol|Filme2| 4.0|
|  Carol|Filme3| 5.0|
+-------+------+----+



In [3]:
# Transforma os nomes em índices numéricos
indexador_usuario = StringIndexer(inputCol="usuario", outputCol="usuarioId")
indexador_item = StringIndexer(inputCol="item", outputCol="itemId")

# Ajusta e transforma os dados
df = indexador_usuario.fit(df).transform(df)
df = indexador_item.fit(df).transform(df)

df.show()


+-------+------+----+---------+------+
|usuario|  item|nota|usuarioId|itemId|
+-------+------+----+---------+------+
|  Alice|Filme1| 5.0|      0.0|   0.0|
|  Alice|Filme2| 3.0|      0.0|   1.0|
|    Bob|Filme1| 4.0|      1.0|   0.0|
|    Bob|Filme3| 2.0|      1.0|   2.0|
|  Carol|Filme2| 4.0|      2.0|   1.0|
|  Carol|Filme3| 5.0|      2.0|   2.0|
+-------+------+----+---------+------+



In [4]:
als = ALS(
    userCol="usuarioId",
    itemCol="itemId",
    ratingCol="nota",
    nonnegative=True,
    coldStartStrategy="drop",
    implicitPrefs=False
)

modelo = als.fit(df)


In [5]:
# Obter os IDs numéricos correspondentes
usuarios_convertidos = df.select("usuario", "usuarioId").distinct()
itens_convertidos = df.select("item", "itemId").distinct()

# Obter os IDs correspondentes a "Alice" e "Filme3"
alice_id = usuarios_convertidos.filter(F.col("usuario") == "Alice").collect()[0]["usuarioId"]
filme3_id = itens_convertidos.filter(F.col("item") == "Filme3").collect()[0]["itemId"]

# Criar um DataFrame com o par para prever
dados_teste = spark.createDataFrame([(alice_id, filme3_id)], ["usuarioId", "itemId"])

previsao = modelo.transform(dados_teste)
previsao.show()

+---------+------+----------+
|usuarioId|itemId|prediction|
+---------+------+----------+
|      0.0|   2.0| 2.9441667|
+---------+------+----------+



In [6]:
avaliador = RegressionEvaluator(
    metricName="rmse",
    labelCol="nota",
    predictionCol="prediction"
)

previsoes_treino = modelo.transform(df)
rmse = avaliador.evaluate(previsoes_treino)
print(f"RMSE: {rmse}")


RMSE: 0.07950812800836858


In [7]:
# Top 2 recomendações por usuário
recomendacoes = modelo.recommendForAllUsers(2)
recomendacoes.show(truncate=False)


+---------+-------------------------------+
|usuarioId|recommendations                |
+---------+-------------------------------+
|0        |[{0, 4.893511}, {1, 2.9873765}]|
|1        |[{0, 3.9027593}, {1, 2.165461}]|
|2        |[{2, 4.872607}, {1, 3.975094}] |
+---------+-------------------------------+

